In [1]:
# Phase 1 – Reading the data
import pandas as pd 
import numpy as np
 
           
clickbait = pd.read_table(" ", header=None, sep='delimiter') # Clickbait file directory 
nonclickbait = pd.read_table(" ", header=None, sep='delimiter')  # non Clickbait file directory


clickbait['label'] = 1
nonclickbait['label'] = 0 

clickbait_df = pd.concat([clickbait, nonclickbait], ignore_index=True)
clickbait_df = clickbait_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_set, val_set, test_set = np.split(clickbait_df, [int(0.72*len(clickbait_df)), int(0.8*len(clickbait_df))])

trainclickbait_target_rate = train_set['label'].mean() * 100
valclickbait_target_rate = val_set['label'].mean() * 100
testclickbait_target_rate = test_set['label'].mean() * 100

print(f"The target rate of the train set is {trainclickbait_target_rate:.2f}%")
print(f"The target rate of the validation set is {valclickbait_target_rate:.2f}%")
print(f"The target rate of the test set is {testclickbait_target_rate:.2f}%")


The target rate of the train set is 35.66%
The target rate of the validation set is 27.23%
The target rate of the test set is 31.17%


C:\Users\Desho\AppData\Local\Temp\ipykernel_62940\2423259785.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  clickbait = pd.read_table("C:\\Users\\Desho\\OneDrive - Illinois Institute of Technology\\Documents\\School IIT\\Fall 2023\\CS 585 Natural Language Processing\\HomeWorks\\HomeWork2\\clickbait.txt", header=None, sep='delimiter')
C:\Users\Desho\AppData\Local\Temp\ipykernel_62940\2423259785.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  nonclickbait = pd.read_table("C:\\Users\\Desho\\OneDrive - Illinois Institute of Technology\\Documents\\School IIT\\Fall 2023\\CS 585 Natural Language Proce

In [34]:
#Phase 2 - Training a single Bag-of-Words (BOW) Text Classifier 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score

clickbait_pipeline =Pipeline ([
    ('vect', CountVectorizer(ngram_range=(1,2))),
    ('clf', MultinomialNB())
])

x_train_set = train_set[0]
y_train_set = train_set['label']

x_val_set = val_set[0]
y_val_set = val_set['label']


clickbait_pipeline.fit(x_train_set,y_train_set)

y_pred_clickbait_train =  clickbait_pipeline.predict(x_train_set)
y_pred_clickbait_val =clickbait_pipeline.predict(x_val_set)


precision_train_set_1 =precision_score(y_train_set, y_pred_clickbait_train, pos_label=1)
recall_train_set_1 =recall_score(y_train_set, y_pred_clickbait_train, pos_label=1)
f1_train_set_1 =f1_score(y_train_set, y_pred_clickbait_train, pos_label=1)


precision_val_set_1 =precision_score(y_val_set, y_pred_clickbait_val, pos_label=1)
recall_val_set_1 =recall_score(y_val_set, y_pred_clickbait_val, pos_label=1)
f1_val_set_1 =f1_score(y_val_set, y_pred_clickbait_val, pos_label=1)


print(" Training set metrics for pos_label =1 for clickbait:")
print(f"Precision: {precision_train_set_1:.2f}")
print(f"Recall: {recall_train_set_1:.2f}")
print(f"F1 Score: {f1_train_set_1:.2f}")

print("\n Validation set metrics pos_label =1 for clickbait:")
print(f"Precision: {precision_val_set_1:.2f}")
print(f"Recall: {recall_val_set_1:.2f}")
print(f"F1 Score: {f1_val_set_1:.2f}")



precision_train_set_0 =precision_score(y_train_set, y_pred_clickbait_train, pos_label=0)
recall_train_set_0 =recall_score(y_train_set, y_pred_clickbait_train, pos_label=0)
f1_train_set_0 =f1_score(y_train_set, y_pred_clickbait_train, pos_label=0)


precision_val_set_0 =precision_score(y_val_set, y_pred_clickbait_val, pos_label=0)
recall_val_set_0 =recall_score(y_val_set, y_pred_clickbait_val, pos_label=0)
f1_val_set_0 =f1_score(y_val_set, y_pred_clickbait_val, pos_label=0)

print(" \n Training set metrics for pos_label =0 for non clickbait:")
print(f"Precision: {precision_train_set_0:.2f}")
print(f"Recall: {recall_train_set_0:.2f}")
print(f"F1 Score: {f1_train_set_0:.2f}")

print("\n Validation set metrics pos_label =0 for non clickbait:")
print(f"Precision: {precision_val_set_0:.2f}")
print(f"Recall: {recall_val_set_0:.2f}")
print(f"F1 Score: {f1_val_set_0:.2f}")


 Training set metrics for pos_label =1 for clickbait:
Precision: 1.00
Recall: 1.00
F1 Score: 1.00

 Validation set metrics pos_label =1 for clickbait:
Precision: 0.94
Recall: 0.90
F1 Score: 0.92
 
 Training set metrics for pos_label =0 for non clickbait:
Precision: 1.00
Recall: 1.00
F1 Score: 1.00

 Validation set metrics pos_label =0 for non clickbait:
Precision: 0.96
Recall: 0.98
F1 Score: 0.97


In [63]:
#Phase 3 - Hyperparameter Tuning

from sklearn.model_selection import ParameterGrid


clickbait_pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])

clickbait_param_grid =  {
    'vect__max_df' : [0.25, 0.5, 0.75,0.80, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0,1.5, 2.0],
    'vect__ngram_range': [(1,1),(1,2)]
}

grid = ParameterGrid(clickbait_param_grid)

results = []
for params in grid:
    clickbait_pipeline2.set_params(**params)
    
    clickbait_pipeline2.fit(x_train_set,y_train_set)

y_pred_clickbait_train2 =  clickbait_pipeline2.predict(x_train_set)
y_pred_clickbait_val2 =clickbait_pipeline2.predict(x_val_set)
    
results_0 = []
for params in grid:
    clickbait_pipeline2.set_params(**params)
    
    clickbait_pipeline2.fit(x_train_set,y_train_set)

y_pred_clickbait_train2 =  clickbait_pipeline2.predict(x_train_set)
y_pred_clickbait_val2 =clickbait_pipeline2.predict(x_val_set)
    
    
new_precision_val_set_1 =precision_score(y_val_set, y_pred_clickbait_val2, pos_label=1)
new_recall_val_set_1 =recall_score(y_val_set, y_pred_clickbait_val2, pos_label=1)
new_f1_val_set_1 =f1_score(y_val_set, y_pred_clickbait_val2, pos_label=1)

new_precision_val_set_0 =precision_score(y_val_set, y_pred_clickbait_val2, pos_label=0)
new_recall_val_set_0 =recall_score(y_val_set, y_pred_clickbait_val2, pos_label=0)
new_f1_val_set_0 =f1_score(y_val_set, y_pred_clickbait_val2, pos_label=0)

results.append({
    
    'params': params,
        'precision': new_precision_val_set_1,
        'recall': new_recall_val_set_1,
        'f1': new_f1_val_set_1
        
    
    
})

results_0.append({
    'params': params,
        'params': params,
        'precision': new_precision_val_set_0,
        'recall': new_recall_val_set_0,
        'f1': new_f1_val_set_0
    
    
})
  



results = sorted(results, key=lambda x: x['f1'], reverse=True)
for result in results:
    print("pos_label =1 for clickbait")
    print(f"Parameters: {result['params']}")
    print(f"Precision: {result['precision']:.2f}")
    print(f"Recall: {result['recall']:.2f}")
    print(f"F1 Score: {result['f1']:.2f}")
    print()
    
    
results_0 = sorted(results_0, key=lambda x: x['f1'], reverse=True)
for result in results_0:
    print("pos_label =0 for non clickbait")
    print(f"Parameters: {result['params']}")
    print(f"Precision: {result['precision']:.2f}")
    print(f"Recall: {result['recall']:.2f}")
    print(f"F1 Score: {result['f1']:.2f}")
    print()
    
    
    
    
    

pos_label =1 for clickbait
Parameters: {'clf__alpha': 2.0, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
Precision: 0.96
Recall: 0.88
F1 Score: 0.92

pos_label =0 for non clickbait
Parameters: {'clf__alpha': 2.0, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
Precision: 0.96
Recall: 0.99
F1 Score: 0.97



In [64]:
# Phase 4 - Model selection

x_test_set = test_set[0]
y_test_set = test_set['label']


clickbait_pipeline2.fit(x_train_set,y_train_set)

y_pred_clickbait_test =  clickbait_pipeline2.predict(x_test_set)

precision_test_set =precision_score(y_test_set, y_pred_clickbait_test, pos_label=1)
recall_test_set =recall_score(y_test_set, y_pred_clickbait_test, pos_label=1)
f1_test_set =f1_score(y_test_set, y_pred_clickbait_test, pos_label=1)

print("Test set metrics:")
print(f"Precision: {precision_test_set:.2f}")
print(f"Recall: {recall_test_set:.2f}")
print(f"F1 Score: {f1_test_set:.2f}")



Test set metrics:
Precision: 0.86
Recall: 0.84
F1 Score: 0.85


In [31]:

#Phase 5 - – Key Indicators

log_prob_best_pipeline = clickbait_pipeline.named_steps['clf'].feature_log_prob_[1, :]

best_pipeline_feature_names = clickbait_pipeline.named_steps['vect'].get_feature_names_out()

best_sorted_features = sorted(zip(log_prob_best_pipeline, best_pipeline_feature_names), reverse=True)

print("Top 5 clickbait indicators:")
for prob, feat in best_sorted_features[:5]:
    print(f"{feat} (log-probability: {prob:.2f})")

Top 5 clickbait indicators:
the (log-probability: -4.78)
you (log-probability: -5.32)
to (log-probability: -5.37)
this (log-probability: -5.45)
is (log-probability: -5.50)


In [32]:
  # Phase 6 - Regular expressions
import re

keywords = []

for prob, feat in best_sorted_features[:5]:
    keywords.append(feat)


pattern = r'\b(?:' + '|'.join(keywords) + r')\b'

regex = re.compile(pattern, re.IGNORECASE)

def check_clickbait(x_test_set):
     match = re.search(regex,x_test_set)
     return bool(match)
     
     


predictions = [check_clickbait(text) for text in x_test_set]
predictions = [int(pred) for pred in predictions]

precision = precision_score(y_test_set, predictions)
recall = recall_score(y_test_set, predictions)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
     

Precision: 0.42
Recall: 0.68
